In [ ]:
import pandas as pd
import numpy as np

from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib

pd.set_option('display.max_columns', None)
Data = pd.read_csv('../data/data_with_merged_cols_garage.csv')
Data

In [ ]:
Data_F = Data.copy()
Data_F

In [ ]:
import re

# Define all possible question patterns and their desired column headers
questions = {
    "Where is the existing garage located\\?": "Where is the existing garage located?",
    "How big is the garage to be converted\\?": "How big is the garage to be converted?",
    "What are you planning to use the converted garage for\\?": "What are you planning to use the converted garage for?",
    "Do you have plans drawn up\\?": "Do you have plans drawn up?",
    "What is the planning permission status for this job\\?": "What is the planning permission status for this job?",
     "How likely are you to make a hiring decision\\?": "How likely are you to make a hiring decision?"

}

# Function to extract answers for each row
def extract_answers(row):
    extracted = {v: None for v in questions.values()}
    pattern = "(" + "|".join(questions.keys()) + ")"
    parts = re.split(pattern, row)

    i = 1
    while i < len(parts) - 1:
        q = parts[i].strip()
        a = parts[i + 1].strip()
        for k, v in questions.items():
            if re.fullmatch(k, q):
                if extracted[v] is None:
                    extracted[v] = a
                elif v == "Additional Details":
                    # Append multiple Additional Details if present
                    extracted[v] += " | " + a
        i += 2

    return pd.Series(extracted)

# Apply the function to your DataFrame
result_df = Data['Que/Ans'].apply(extract_answers)
# Clean encoding artifacts (like Â, â€¦, etc.)
result_df = result_df.replace({r'[^\x00-\x7F]+': ''}, regex=True)

# Preview the cleaned DataFrame
result_df.head()

In [ ]:
Data = pd.concat([Data, result_df], axis=1)

In [ ]:
columns_to_drop = [
    'Time','Que/Ans','Category','Additional Details'
]

# Drop the columns (if they exist in the DataFrame)
Data = Data.drop(columns=[col for col in columns_to_drop if col in Data.columns])

pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', None)   # Show all rows


df = pd.DataFrame(Data)
df.to_csv('../data/sample_data_for_prediction_garage.csv', index=False)

df
